In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada.csv', sep = '|')
df.head()

,acordao,arquivo,texto,areas
0,297/2016-P,547240.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 010.084/2015-0...,Responsabilidade
1,366/2016-P,549518.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 005.933/2014-5...,Finanças Públicas
2,944/2016-P,554399.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 042.038/2012-0...,Responsabilidade
3,30/2016-P,545010.txt,TRIBUNAL DE CONTAS DA UNIÃO\tTC 000.742/2014-7...,Direito Processual
4,55/2016-P,544046.txt,;-;;Wania Lucia Pasquarelli do NascimentoTCUWa...,Pessoal


In [2]:
df.shape

(10524, 4)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Competência do TCU,Contrato Administrativo',
       'Competência do TCU,Contrato Administrativo,Responsabilidade',
       'Competência do TCU,Convênio',
       'Competência do TCU,Convênio,Direito Processual,Licitação',
       'Competência do TCU,Convênio,Gestão Administrativa',
       'Competência do TCU,Convênio,Responsabilidade',
       'Competência do TCU,Desestatização',
       'Competência do TCU,Direito Processual',
       'Competência do TCU,Direito Processual,Licitação',
       'Competência do TCU,Direito Processual,Pessoal',
       'Competência do TCU,Direito Processual,Responsabilidade',
       'Competência do TCU,Finanças Públicas',
       'Competência do TCU,Finanças Públicas,Licitação',
       'Competência do TCU,Finanças Públicas,Responsabilidade',
       'Competência do TCU,Gestão Administrativa',
       'Competência do TCU,Licitação',
       'Competência do TCU,Licitação,Pessoal',
       'Competência do TCU,Licitação,Responsabilidade',
   

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(10524, 91)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 350000
limite_texto = 40000
dim_vetor = 50

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['texto'])

sequences = tokenizer.texts_to_sequences(df['texto'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 344296 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (10524, 40000)


In [7]:
x.shape, y.shape

((10524, 40000), (10524, 91))

In [8]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0309 22:56:46.857789 140614950025024 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0309 22:56:46.869385 140614950025024 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0309 22:56:46.870996 140614950025024 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0309 22:56:46.902403 140614950025024 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is d

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40000, 50)         17500000  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2000000)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                128000064 
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              263168    
__________

W0309 22:56:47.886842 140614950025024 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 8419 samples, validate on 2105 samples
Epoch 1/20
8419/8419 [==============================] - 60s 7ms/step - loss: 2.5215 - categorical_accuracy: 0.3114 - val_loss: 1.7953 - val_categorical_accuracy: 0.4575
Epoch 2/20
8419/8419 [==============================] - 55s 6ms/step - loss: 1.6763 - categorical_accuracy: 0.5377 - val_loss: 1.7617 - val_categorical_accuracy: 0.5121
Epoch 3/20
8419/8419 [==============================] - 55s 7ms/step - loss: 1.2787 - categorical_accuracy: 0.6419 - val_loss: 1.6946 - val_categorical_accuracy: 0.4879
Epoch 4/20
8419/8419 [==============================] - 55s 7ms/step - loss: 0.9800 - categorical_accuracy: 0.7436 - val_loss: 1.9554 - val_categorical_accuracy: 0.3762
Epoch 5/20
8419/8419 [==============================] - 55s 7ms/step - loss: 0.7859 - categorical_accuracy: 0.8109 - val_loss: 1.9853 - val_categorical_accuracy: 0.5525
Epoch 6/20
8419/8419 [==============================] - 55s 7ms/step - loss: 0.6223 - categorical_accuracy: